In [9]:
import os

import pdfplumber
from bs4 import BeautifulSoup
from pdf2image import convert_from_path
from pdfplumber.utils import extract_text, get_bbox_overlap, obj_to_bbox

import pandas as pd

In [12]:
from pydantic_settings import BaseSettings, SettingsConfigDict

class Settings(BaseSettings):
    model_config = SettingsConfigDict(
        env_file="../.env", env_file_encoding="utf-8", extra="ignore"
    )
    data_dir: str
    docling_model_dir: str
    
settings = Settings()
os.environ["HF_HOME"] = settings.docling_model_dir

In [13]:
# source = "https://arxiv.org/pdf/2408.09869"  # document per local path or URL

# fname = "1706.03762v7"
# fname = "1706.03762v7_sample"
# # fname = "ai_parl_2" # 국회도서관 문서 - 이미지 PDF
# # fname = "ai_parl_3" # 국회도서관 문서 - 텍스트 PDF

# source = f"samples/{fname}.pdf" # attention is all you need

pdf_dir = os.path.join(settings.data_dir, "allganize-RAG-Evaluation-Dataset-KO/finance")
fname = "★2019 제1회 증시콘서트 자료집_최종★"

source = os.path.join(pdf_dir, f"{fname}.pdf")

result_dir = f"results/pdfplumber/{fname}"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

In [14]:
all_pages = []
with pdfplumber.open(source) as pdf:
    for page in pdf.pages:
        filtered_page = page
        chars = filtered_page.chars

        for table in page.find_tables():
            try:
                first_table_char = page.crop(table.bbox).chars[0]
            except Exception:
                continue
            filtered_page = filtered_page.filter(
                lambda obj: get_bbox_overlap(obj_to_bbox(obj), table.bbox) is None
            )
            chars = filtered_page.chars
            df = pd.DataFrame(table.extract())
            df.columns = df.iloc[0]
            html = df.drop(0).to_markdown(index=False)
            chars.append(first_table_char | {"text": html})
        page_text = extract_text(chars, layout=True)
        # all_pages.extend(self._parse_content(page_text))
        all_pages.append(
            {
                "text": page_text,
                "italicEndOffsets": [],
                "notes": [],
                "citations": [],
                "indent": False,
                "memos": [],
                "italicStartOffsets": [],
                "type": "plain",
                "captions": [],
            }
        )

In [20]:
# Text has (cid:x) patterns
# ex. "(cid:1523)(cid:1758)..."
all_pages[-10:-8]

[{'text': "(cid:11407)(cid:4943)(cid:1383)(cid:1012)(cid:8019)(cid:7995)(cid:4372)\n(cid:3299)(cid:6279)(cid:6231)(cid:35)(cid:3983)(cid:6279)(cid:4463)(cid:35)(cid:2910)(cid:9755)(cid:35)(cid:2968)(cid:3339)(cid:11587)(cid:11151)(cid:6231)(cid:35)(cid:9779)(cid:6867)(cid:66)(cid:35)\n\n(cid:10067)(cid:9822)(cid:35)(cid:10431)(cid:10242)(cid:9850)(cid:35)(cid:10662)(cid:10242)(cid:61)(cid:35)(cid:3431)(cid:3802)(cid:12050)(cid:11614)(cid:4926)(cid:35)(cid:53)(cid:3270)(cid:35)(cid:7726)(cid:3802)(cid:35)(cid:9882)(cid:8759)(cid:10214)(cid:6406)(cid:35)(cid:74)(cid:71)(cid:83)(cid:35)(cid:8667)(cid:10319)(cid:6666)(cid:10270)(cid:35)(cid:6770)(cid:10270)(cid:4530)(cid:9038)(cid:6694)(cid:35)(cid:3802)(cid:6407)(cid:35)\n(cid:53)(cid:51)(cid:51)(cid:52)(cid:4590)(cid:3494)(cid:35)(cid:53)(cid:51)(cid:51)(cid:58)(cid:4590)(cid:9850)(cid:4926)(cid:35)(cid:9882)(cid:10666)(cid:10270)(cid:35)(cid:3762)(cid:6742)(cid:6694)(cid:35)(cid:4446)(cid:6746)(cid:35)(cid:9094)(cid:10426)(cid:10270)(ci

In [16]:
import re
def prune_text(text):
    def replace_cid(match):
        ascii_num = int(match.group(1))
        try:
            return chr(ascii_num)
        except:
            return ''  # In case of conversion failure, return empty string

    # Regular expression to find all (cid:x) patterns
    cid_pattern = re.compile(r'\(cid:(\d+)\)')
    pruned_text = re.sub(cid_pattern, replace_cid, text)
    return pruned_text

text = '(cid:42)(cid:42)(cid:42)(cid:15)(cid:1)(cid:3294)(cid:1992)(cid:1245)(cid:1)(cid:2681)(cid:1870)(cid:1175)(cid:1)(cid:2681)(cid:1754)(cid:1)\n(cid:1523)(cid:1758)(cid:1)(cid:1098)(cid:2241)(cid:2496)(cid:1)(cid:2689)(cid:1586)(cid:27)(cid:1)(cid:2507)(cid:1864)(cid:1)(cid:2583)(cid:16)(cid:1523)(cid:1758)(cid:1)(cid:3356)(cid:2607)(cid:1)(cid:18)(cid:13)(cid:18)(cid:22)(cid:17)(cid:2583)(cid:1)(cid:2681)(cid:1870)(cid:1)\n\n\n\n\n(cid:2583)(cid:3354)(cid:1)(cid:2680)(cid:2687)(cid:1)(cid:2299)(cid:2739)(cid:2613)(cid:1789)(cid:1)(cid:2050)(cid:1224)(cid:1) (cid:2583)(cid:3354)(cid:2540)(cid:1)(cid:2596)(cid:2441)(cid:3354)(cid:1)(cid:1088)(cid:1)(cid:1586)(cid:2705)(cid:3354)(cid:1)(cid:2348)(cid:3354)(cid:1)\n\n(cid:9)(cid:2769)(cid:2299)(cid:27)(cid:1)(cid:18)(cid:26)(cid:26)(cid:21)(cid:1433)(cid:1)(cid:30)'
prune_text(text)

'***\x0f\x01ೞ߈ӝ\x01\u0a79ݎҗ\x01\u0a79ۚ\x01\n׳۞\x01ъࣁী\x01ઁز\x1b\x01ো݈\x01ਗ\x10׳۞\x01ജਯ\x01\x12\r\x12\x16\x11ਗ\x01\u0a79ݎ\x01\n\n\n\n\nਗച\x01\u0a78\u0a7f\x01ࣻળਵ۽\x01ࠂӈ\x01 ਗച৬\x01ਤউച\x01р\x01زઑച\x01बച\x01\n\n\t\u0ad1ࣻ\x1b\x01\x12\x1a\x1a\x15֙\x01\x1e'